In [1]:
import numpy as np
import time
import pickle
import os
import matplotlib.pyplot as plt
import warnings
import pandas as pd

warnings.filterwarnings('ignore')

In [2]:
print("----------------Reading the Data-------------------------")
PATH = os.getcwd()
os.chdir('Alphabets/')

X_train = pd.read_csv('train.csv', sep=',', header=None, index_col=False)
X_test = pd.read_csv('test.csv', sep=',', header=None, index_col=False)
np.random.shuffle(X_train.to_numpy())
train_class = X_train[X_train.columns[-1]]
test_actual_class = X_test[X_test.columns[-1]]

X_train = X_train.drop(X_train.columns[-1], axis=1)
X_test = X_test.drop(X_test.columns[-1], axis=1)

print("----------------Data Reading completed-------------------")

os.chdir('../')

X_train = X_train/255
X_test = X_test/255

m = X_train.shape[0] # Number of Training Samples

X_valid = X_train.iloc[(int(0.85*m)):]
valid_class = train_class[(int(0.85*m)):]
X_train = X_train.iloc[0:int(0.85*m)]
train_class = train_class[0:int(0.85*m)]


m = X_train.shape[0] # Number of Training Samples
n = X_train.shape[1] # Number of input features

print("The total number of training samples = {}".format(m))
print("The total number of validation samples = {}".format(X_valid.shape[0]))

print("The number of features = {}".format(n))

----------------Reading the Data-------------------------
----------------Data Reading completed-------------------
The total number of training samples = 11050
The total number of validation samples = 1950
The number of features = 784


In [3]:
#To get the one hot encoding of each label
print("--------Perform 1-hot encoding of class labels------------")

train_class_enc = pd.get_dummies(train_class).to_numpy()
valid_class_enc = pd.get_dummies(valid_class).to_numpy()
test_actual_class_enc = pd.get_dummies(test_actual_class).to_numpy()

--------Perform 1-hot encoding of class labels------------


In [4]:
#Add the intercept term to the data samples both in training and test dataset
X_train = np.hstack((np.ones((m,1)),X_train.to_numpy()))
X_valid = np.hstack((np.ones((X_valid.shape[0],1)), X_valid.to_numpy()))
X_test = np.hstack((np.ones((X_test.shape[0],1)),X_test.to_numpy()))

In [5]:
lr = 0.1
arch_test = [1,5,10,50,100]
arch = [arch_test[3]] #means one hidden layer with 2 perceptrons 
batch_size = 100 # Mini-Batch Size
r = np.max(train_class) + 1 # Default value of the number of classes = 26

In [6]:
#Mini-Batch formation
mini_batch = [(X_train[i:i+batch_size,:], train_class_enc[i:i+batch_size]) for i in range(0, m, batch_size)]
print("The number of mini-batches formed is = {}".format(len(mini_batch)))

The number of mini-batches formed is = 111


In [7]:
#Theta Initialization 
def theta_init(arch=[50]):
    theta = []
    for i in range(len(arch)+1):
        if i == 0:
            dim0=n+1
            dim1=arch[i]
        elif (i == len(arch)):
            dim0=arch[i-1]
            dim1 = r
        else:
            dim0=arch[i-1]
            dim1= arch[i]

        theta.append(2*np.random.random((dim0, dim1))-1)
        #theta.append(np.zeros((dim0, dim1)))
    return theta

In [8]:
def activation(x):
    return 1/(1+np.exp(-x))

In [9]:
def forward_prop(data, theta):
    fm = []
    fm.append(data)
    for l in range(len(theta)):
        fm.append(activation(np.dot(fm[l], theta[l])))
    return fm

In [10]:
def cost_total(X, theta, Y, m):
    fm = forward_prop(X, theta)
    cost = (1/(2*m))*np.sum((Y-fm[-1])**2)
    return cost

In [11]:
def calc_accuracy(data, theta, actual_class):
    pred_class = forward_prop(data, theta)
    test_pred_class = pred_class[-1]
    for i in range(len(test_pred_class)):
        test_pred_class[i][test_pred_class[i] == np.max(test_pred_class[i])] = 1
        test_pred_class[i][test_pred_class[i] != np.max(test_pred_class[i])] = 0


    test_acc = 0
    for i in range(len(actual_class)):
        if (np.array_equal(test_pred_class[i], actual_class[i])):
            test_acc+=1
    test_acc /= data.shape[0]

    #print("The Test Accuracy of the model = {}%".format(test_acc*100))
    return (test_acc*100)

In [12]:
epochs = []
train_accuracy = []
test_accuracy = []
valid_accuracy=[]
train_time = []

In [28]:
lr0=1.5
arch_test=[100]
for i in range(len(arch_test)):
    theta = theta_init([arch_test[i]])
    fm = forward_prop(X_train, theta)
    ite=1
    epoch = 0
    start = time.time()
    cost_init = cost_total(X_valid, theta, valid_class_enc, m)

    while(True):
        count = 0
        lr = lr0/(np.sqrt(ite))
        print("learning rate = ", lr)

        print("Initial Cost on Val dataset for this epoch {} = {}".format(epoch, cost_init))

        for b in mini_batch:
            #if (lr < 0.1): lr = 0.1
            X_b = b[0]
            Y_b = b[1]
            fm = forward_prop(X_b, theta)
            delta = [None]*len(fm)

            if (count % 60 == 0):
                print("Error on this batch = "+str(cost_total(X_b, theta, Y_b, batch_size)))
            #Backward Propagation

            for l in range(len(fm)-1, 0, -1):
                if (l == len(fm)-1):
                    delta[l] = ((1/batch_size)*(Y_b - fm[l])*fm[l]*(1-fm[l]))
                else:
                    delta[l]=(np.dot(delta[l+1], theta[l].T)*fm[l]*(1-fm[l]))

            #delta_l1 = np.dot(delta_l2, theta2.T)*l1*(1-l1)

            for t in range(len(theta)):
                theta[t] += lr*np.dot(fm[t].T, delta[t+1]) 

            count+=1
        epoch+=1 #Number of epochs
        ite+=1
        
        cost_final = cost_total(X_valid, theta, valid_class_enc, m)
        print("Cost on val dataset after {} epochs is = {}".format(epoch, cost_final))
        if (abs(cost_final-cost_init) < 1e-06):
            print("cost initial= {} , cost final={} , change in cost= {}".format(cost_init,cost_final, cost_final-cost_init))
            break
        cost_init = cost_final
    epochs.append(epoch)
    train_time.append(time.time()-start)
    train_accuracy.append(calc_accuracy(X_train, theta, train_class_enc))
    valid_accuracy.append(calc_accuracy(X_valid, theta, valid_class_enc))
    test_accuracy.append(calc_accuracy(X_test, theta, test_actual_class_enc))
    print("\n------------------------------------------------------------------------------")
    print("The stats for number of units in the hidden layer = {} are as below:".format(arch_test[i]))
    print("------------------------------------------------------------------------------")
    print("The number of epochs = {:2.3f}".format(epochs[-1]))
    print("The training time = {:2.3f}sec".format(train_time[-1]))
    print("The training accuracy is = {:2.3f}%".format(train_accuracy[-1]))
    print("The validation accuracy is = {:2.3f}%".format(valid_accuracy[-1]))
    print("The test accuracy is = {:2.3f}%".format(test_accuracy[-1]))
    print("------------------------------------------------------------------------------\n")

learning rate =  1.5
Initial Cost on Val dataset for this epoch 0 = 1.0357980972016674
Error on this batch = 5.915096418485747
Error on this batch = 0.4912251055742077
Cost on val dataset after 1 epochs is = 0.08449712773923045
learning rate =  1.0606601717798212
Initial Cost on Val dataset for this epoch 1 = 0.08449712773923045
Error on this batch = 0.48298644685510733
Error on this batch = 0.47827137077464166
Cost on val dataset after 2 epochs is = 0.08109410749997038
learning rate =  0.8660254037844387
Initial Cost on Val dataset for this epoch 2 = 0.08109410749997038
Error on this batch = 0.4640634340097464
Error on this batch = 0.4596095132636519
Cost on val dataset after 3 epochs is = 0.07829379955419291
learning rate =  0.75
Initial Cost on Val dataset for this epoch 3 = 0.07829379955419291
Error on this batch = 0.45363469079923124
Error on this batch = 0.4542045800812817
Cost on val dataset after 4 epochs is = 0.07698173061020931
learning rate =  0.6708203932499369
Initial Cost

Error on this batch = 0.3737773541871235
Cost on val dataset after 35 epochs is = 0.06322419395126803
learning rate =  0.25
Initial Cost on Val dataset for this epoch 35 = 0.06322419395126803
Error on this batch = 0.37724772034688303
Error on this batch = 0.37169592231877757
Cost on val dataset after 36 epochs is = 0.0629438336129316
learning rate =  0.24659848095803594
Initial Cost on Val dataset for this epoch 36 = 0.0629438336129316
Error on this batch = 0.3745868247421224
Error on this batch = 0.36935403735247635
Cost on val dataset after 37 epochs is = 0.06264421875086282
learning rate =  0.24333213169614382
Initial Cost on Val dataset for this epoch 37 = 0.06264421875086282
Error on this batch = 0.3716847972261598
Error on this batch = 0.3667346420693373
Cost on val dataset after 38 epochs is = 0.062322675566374046
learning rate =  0.24019223070763068
Initial Cost on Val dataset for this epoch 38 = 0.062322675566374046
Error on this batch = 0.3685301000895905
Error on this batch 

Cost on val dataset after 69 epochs is = 0.053705852558991525
learning rate =  0.17928429140015903
Initial Cost on Val dataset for this epoch 69 = 0.053705852558991525
Error on this batch = 0.3065348085613996
Error on this batch = 0.3027857848229039
Cost on val dataset after 70 epochs is = 0.05352389973349451
learning rate =  0.17801724872907798
Initial Cost on Val dataset for this epoch 70 = 0.05352389973349451
Error on this batch = 0.305359131241745
Error on this batch = 0.3011994762433208
Cost on val dataset after 71 epochs is = 0.05334481907534534
learning rate =  0.1767766952966369
Initial Cost on Val dataset for this epoch 71 = 0.05334481907534534
Error on this batch = 0.3042096163898966
Error on this batch = 0.29957327997148925
Cost on val dataset after 72 epochs is = 0.05316924518958036
learning rate =  0.17556172079419585
Initial Cost on Val dataset for this epoch 72 = 0.05316924518958036
Error on this batch = 0.303085129669191
Error on this batch = 0.29793070373248626
Cost on

Cost on val dataset after 102 epochs is = 0.04982541541507088
learning rate =  0.14779939172464399
Initial Cost on Val dataset for this epoch 102 = 0.04982541541507088
Error on this batch = 0.2809045823893061
Error on this batch = 0.2668758282049775
Cost on val dataset after 103 epochs is = 0.04975082323407731
learning rate =  0.14708710135363803
Initial Cost on Val dataset for this epoch 103 = 0.04975082323407731
Error on this batch = 0.2804016669565274
Error on this batch = 0.26613281097237085
Cost on val dataset after 104 epochs is = 0.049677308575865234
learning rate =  0.14638501094227999
Initial Cost on Val dataset for this epoch 104 = 0.049677308575865234
Error on this batch = 0.27990355591511806
Error on this batch = 0.265398324946545
Cost on val dataset after 105 epochs is = 0.04960479634557179
learning rate =  0.14569287935358963
Initial Cost on Val dataset for this epoch 105 = 0.04960479634557179
Error on this batch = 0.2794093119820123
Error on this batch = 0.26467231728236

Cost on val dataset after 135 epochs is = 0.047365484440745044
learning rate =  0.12862393885688161
Initial Cost on Val dataset for this epoch 135 = 0.047365484440745044
Error on this batch = 0.25626241788588183
Error on this batch = 0.24539759024271626
Cost on val dataset after 136 epochs is = 0.04726689558729737
learning rate =  0.12815364865751414
Initial Cost on Val dataset for this epoch 136 = 0.04726689558729737
Error on this batch = 0.25520802077844307
Error on this batch = 0.24459588287237702
Cost on val dataset after 137 epochs is = 0.047166462475456444
learning rate =  0.1276884796138123
Initial Cost on Val dataset for this epoch 137 = 0.047166462475456444
Error on this batch = 0.25423168390450956
Error on this batch = 0.2437737697019469
Cost on val dataset after 138 epochs is = 0.04706449541151074
learning rate =  0.12722833945199566
Initial Cost on Val dataset for this epoch 138 = 0.04706449541151074
Error on this batch = 0.25334165130935826
Error on this batch = 0.24294137

...

Cost on val dataset after 2408 epochs is = 0.02788176543657189
learning rate =  0.030561372831402304
Initial Cost on Val dataset for this epoch 2408 = 0.02788176543657189
Error on this batch = 0.12989629453340645
Error on this batch = 0.10260359281821818
Cost on val dataset after 2409 epochs is = 0.02788055756307021
learning rate =  0.030555031639751644
Initial Cost on Val dataset for this epoch 2409 = 0.02788055756307021
Error on this batch = 0.12988819737995194
Error on this batch = 0.10258955459170586
Cost on val dataset after 2410 epochs is = 0.02787935165140353
learning rate =  0.03054869439367296
Initial Cost on Val dataset for this epoch 2410 = 0.02787935165140353
Error on this batch = 0.1298801018354974
Error on this batch = 0.10257552465118963
Cost on val dataset after 2411 epochs is = 0.02787814769225509
learning rate =  0.030542361089076306
Initial Cost on Val dataset for this epoch 2411 = 0.02787814769225509
Error on this batch = 0.12987200792077633
Error on this batch = 0.

Cost on val dataset after 2441 epochs is = 0.027842893263302405
learning rate =  0.030354174694363235
Initial Cost on Val dataset for this epoch 2441 = 0.027842893263302405
Error on this batch = 0.12963004193429944
Error on this batch = 0.10214451674145356
Cost on val dataset after 2442 epochs is = 0.027841745557407892
learning rate =  0.03034796157901939
Initial Cost on Val dataset for this epoch 2442 = 0.027841745557407892
Error on this batch = 0.12962200761325737
Error on this batch = 0.10213073398332273
Cost on val dataset after 2443 epochs is = 0.02784059953860219
learning rate =  0.03034175227735256
Initial Cost on Val dataset for this epoch 2443 = 0.02784059953860219
Error on this batch = 0.12961397547068923
Error on this batch = 0.10211695843458582
Cost on val dataset after 2444 epochs is = 0.027839455199515518
learning rate =  0.030335546785462863
Initial Cost on Val dataset for this epoch 2444 = 0.027839455199515518
Error on this batch = 0.12960594552046995
Error on this batc

Cost on val dataset after 2474 epochs is = 0.027805871517801936
learning rate =  0.03015113445777636
Initial Cost on Val dataset for this epoch 2474 = 0.027805871517801936
Error on this batch = 0.1293661282413389
Error on this batch = 0.10169334550005399
Cost on val dataset after 2475 epochs is = 0.027804775854375918
learning rate =  0.03014504516468028
Initial Cost on Val dataset for this epoch 2475 = 0.027804775854375918
Error on this batch = 0.12935817221113677
Error on this batch = 0.10167978676303836
Cost on val dataset after 2476 epochs is = 0.027803681659402467
learning rate =  0.030138959559457335
Initial Cost on Val dataset for this epoch 2476 = 0.027803681659402467
Error on this batch = 0.1293502187298794
Error on this batch = 0.10166623440616157
Cost on val dataset after 2477 epochs is = 0.027802588926977893
learning rate =  0.030132877638386544
Initial Cost on Val dataset for this epoch 2477 = 0.027802588926977893
Error on this batch = 0.1293422678062347
Error on this batch

learning rate =  0.029958087994702943
Initial Cost on Val dataset for this epoch 2506 = 0.027771510682226778
Error on this batch = 0.1291128379120271
Error on this batch = 0.10126252947415601
Cost on val dataset after 2507 epochs is = 0.02777045926132952
learning rate =  0.029952114893657688
Initial Cost on Val dataset for this epoch 2507 = 0.02777045926132952
Error on this batch = 0.12910496711774205
Error on this batch = 0.10124916504807678
Cost on val dataset after 2508 epochs is = 0.027769409137614807
learning rate =  0.02994614536397336
Initial Cost on Val dataset for this epoch 2508 = 0.027769409137614807
Error on this batch = 0.1290970990883468
Error on this batch = 0.10123580639761008
Cost on val dataset after 2509 epochs is = 0.027768360306270102
learning rate =  0.029940179402092468
Initial Cost on Val dataset for this epoch 2509 = 0.027768360306270102
Error on this batch = 0.12908923382873902
Error on this batch = 0.10122245350671198
Cost on val dataset after 2510 epochs is 

Error on this batch = 0.12886236475076904
Error on this batch = 0.10083765870176922
Cost on val dataset after 2539 epochs is = 0.02773747361888094
learning rate =  0.029762842129970442
Initial Cost on Val dataset for this epoch 2539 = 0.02773747361888094
Error on this batch = 0.12885458439760025
Error on this batch = 0.10082447192279446
Cost on val dataset after 2540 epochs is = 0.027736462618190003
learning rate =  0.02975698503218078
Initial Cost on Val dataset for this epoch 2540 = 0.027736462618190003
Error on this batch = 0.12884680692079897
Error on this batch = 0.10081129048357976
Cost on val dataset after 2541 epochs is = 0.027735452770484066
learning rate =  0.029751131390925722
Initial Cost on Val dataset for this epoch 2541 = 0.027735452770484066
Error on this batch = 0.12883903232249636
Error on this batch = 0.10079811437296032
Cost on val dataset after 2542 epochs is = 0.027734444071826975
learning rate =  0.029745281202806846
Initial Cost on Val dataset for this epoch 254

In [17]:
calc_accuracy(X_test, theta, test_actual_class_enc)

84.3076923076923

In [27]:
test_accuracy

[84.07692307692308, 73.9076923076923, 76.58461538461539]

In [25]:
epochs

[2046, 1459]